# 接口：从协议到抽象基类

In [3]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [4]:
f = Foo()
f[1]

10

In [5]:
for i in f:
    print(i)

0
10
20


In [6]:
20 in f

True

In [7]:
15 in f

False

In [8]:
import collections


Card = collections.namedtuple('Card', ['rand', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):  # doctest: +ELLIPSIS
        return len(self._cards)

    def __getitem__(self, position):  # doctest: +ELLIPSIS
        return self._cards[position]

In [9]:
from random import shuffle

l = list(range(10))
shuffle(l)
l

[2, 8, 0, 3, 5, 9, 7, 4, 1, 6]

In [10]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [11]:
def set_card(deck, position, card):    # 在运行时实现协议(猴子补丁)
    deck._cards[position] = card
    
    
FrenchDeck.__setitem__ = set_card

In [12]:
shuffle(deck)

In [13]:
deck[:5]

[Card(rand='J', suit='spades'),
 Card(rand='2', suit='diamonds'),
 Card(rand='7', suit='clubs'),
 Card(rand='A', suit='spades'),
 Card(rand='5', suit='clubs')]

In [14]:
field_name = 'rand, suit'
try:
    field_names = field_name.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)
field_names

('rand', 'suit')

In [15]:
from collections.abc import MutableSequence

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

In [16]:
import abc


class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回。如果实例为空，这个方法应该抛出`LookupError`。"""
        
    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        """返回一个有序元组，由当前元素构成。"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [33]:
class Fake(Tombola):
    def pick(self):
        return 13

In [34]:
Fake

__main__.Fake

In [35]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [20]:
import random


class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [21]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
                               
    def loaded(self):
        return bool(self._balls)
                               
    def inspect(self):
        return tuple(sorted(self._balls))

In [22]:
from random import randrange


@Tombola.register
class TomboList(list):    # 为了避免运行时错误，虚拟子类要实现所需的全部方法
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
    
# Tombola.register(TomboList)

In [23]:
issubclass(TomboList, Tombola)

True

In [24]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [25]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [26]:
from random import shuffle


class TumblingDrum(Tombola):

    def __init__(self, iterable):
        self._balls = []
        self.load(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)
        shuffle(self._balls)

    def pick(self):
        return self._balls.pop()

In [27]:
class Struggle:
    def __len__(self):
        return 23

In [28]:
from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [29]:
issubclass(Struggle, abc.Sized)

True